In [1]:
import pandas as pd, json

In [2]:
datasets = ['BGP_testbed_2','BGP_testbed_3','BGP_testbed_5','BGP_testbed_9', 'BGP_testbed_10']

config = json.loads(open('../../configuration.json').read())

methods = ['dbscan', 'lof', 'wdbscan', 'exactSTORM', 'COD', 'rrcf', 'ods']
features_set = 'ALL'

In [3]:
def read_results_dataset(methods):
    res = {}
    
    for method in methods:
        print(method)
        res[method] = pd.read_json('../../Experiments/'+method+'/Results/'+method+'_results_test_'+features_set+'.json').astype(float)
    return res

def read_results_general_scores(methods):
    res = {}
    for method in methods:
        res[method] = json.loads(open('../../Experiments/'+method+'/Results/'+method+'_general_results_test_'+features_set+'.json').read())
    return res

def read_results_general_scores_tune(methods):
    res = {}
    for method in methods:
        res[method] = json.loads(open('../../Experiments/'+method+'/Results/'+method+'_general_results_tune_'+features_set+'.json').read())
    return res

In [4]:
def get_dataset_methods_result(methods_data, dataset_name):
    
    result = {}
    result['dataset_name'] = dataset_name
    result['scores'] = []
    
    for method in methods:
        method_scores = []
        method_scores.append(method)
        method_scores.append(round(methods_data[method].loc['avg_prec', dataset_name],2))
        method_scores.append(round(methods_data[method].loc['ci_prec_low', dataset_name],2))
        
        method_scores.append(round(methods_data[method].loc['avg_rec', dataset_name],2))
        method_scores.append(round(methods_data[method].loc['ci_rec_low', dataset_name],2))
        
        method_scores.append(round(methods_data[method].loc['avg_fscore', dataset_name],2))
        method_scores.append(round(methods_data[method].loc['ci_fscore_low', dataset_name],2))
                
        result['scores'].append(method_scores)
        
    return result

def add_total(methods_data, dataset_name, current_result, metrics = 'PRF'):
    
    result = {}
    result['dataset_name'] = dataset_name
    result['scores'] = []
    
    for method in methods:
        method_scores = []
        method_scores.append(method)
        
        if metrics == 'PRF':
            
            method_scores.append(round(methods_data[method]['precision'],4))
            method_scores.append(round(methods_data[method]['ci_precision'],4))     

            method_scores.append(round(methods_data[method]['recall'],4))
            method_scores.append(round(methods_data[method]['ci_recall'],4))  

            method_scores.append(round(methods_data[method]['fscore'],4))
            method_scores.append(round(methods_data[method]['ci_fscore'],4))
            
        elif metrics == 'AMI':
                
            method_scores.append(round(methods_data[method]['ACC'],3))
            method_scores.append(round(methods_data[method]['ci_ACC'],3))       

            method_scores.append(round(methods_data[method]['MK'],3))
            method_scores.append(round(methods_data[method]['ci_MK'],3))               

            method_scores.append(round(methods_data[method]['BM'],3))
            method_scores.append(round(methods_data[method]['ci_BM'],3))
        
        else:
            print('UNKNOWN metrics')
                        
        result['scores'].append(method_scores)
        
    current_result.append(result)

def get_results(methods_data, metrics):
    res = []
#     for dataset in datasets:
#         res.append(get_dataset_methods_result(methods_data, dataset))
        
    methods_data_test = read_results_general_scores(methods)
    methods_data_tune = read_results_general_scores_tune(methods)

    add_total(methods_data_test, 'test', res, metrics)
    add_total(methods_data_tune, 'tune', res, metrics)

    return res

In [5]:
"""
Please choose the metrics to parse. 

metrics == 'PRF' to produce Precision-Recall-Fscore results (Fig. 8)
metrics = 'AMI' produce Accuracy-Markedness-Informedness results (Fig. 9)
"""

metrics = 'PRF'
# metrics = 'AMI'

methods_data_dataset = read_results_dataset(methods)
res = get_results(methods_data_dataset, metrics = metrics)
res

dbscan
lof
wdbscan
exactSTORM
COD
rrcf
ods


[{'dataset_name': 'test',
  'scores': [['dbscan', 0.9128, 0.0584, 0.5924, 0.0817, 0.7931, 0.0604],
   ['lof', 0.7101, 0.1044, 0.7758, 0.0768, 0.7016, 0.0918],
   ['wdbscan', 1.0, nan, 0.351, 0.0697, 0.7098, 0.0526],
   ['exactSTORM', 1.0, nan, 0.3813, 0.0779, 0.7277, 0.0573],
   ['COD', 0.7332, 0.0852, 0.3606, 0.0746, 0.5855, 0.0497],
   ['rrcf', 0.8791, 0.0648, 0.7596, 0.0787, 0.8284, 0.0573],
   ['ods', 0.9575, 0.0314, 0.5419, 0.0785, 0.7867, 0.0511]]},
 {'dataset_name': 'tune',
  'scores': [['dbscan', 0.9852, 0.0218, 0.8839, 0.0489, 0.9595, 0.0188],
   ['lof', 0.9885, 0.0167, 0.8912, 0.0469, 0.9647, 0.0168],
   ['wdbscan', 1.0, nan, 0.6964, 0.0691, 0.9106, 0.0266],
   ['exactSTORM', 1.0, nan, 0.7037, 0.0677, 0.9141, 0.0241],
   ['COD', 0.9015, 0.0631, 0.7151, 0.0647, 0.8488, 0.0556],
   ['rrcf', 0.9607, 0.0463, 0.8369, 0.0593, 0.9266, 0.0407],
   ['ods', 1.0, nan, 0.6347, 0.0806, 0.8814, 0.0351]]}]

In [6]:
import json
with open('total_results_'+features_set+'_'+metrics+'.json', 'w') as f:
    json.dump(res, f, indent=2)

In [7]:
"""
Please choose the metrics to parse. 

metrics == 'PRF' to produce Precision-Recall-Fscore results (Fig. 8)
metrics = 'AMI' produce Accuracy-Markedness-Informedness results (Fig. 9)
"""

# metrics = 'PRF'
metrics = 'AMI'

methods_data_dataset = read_results_dataset(methods)
res = get_results(methods_data_dataset, metrics = metrics)
res

dbscan
lof
wdbscan
exactSTORM
COD
rrcf
ods


[{'dataset_name': 'test',
  'scores': [['dbscan', 0.986, 0.003, 0.901, 0.059, 0.59, 0.082],
   ['lof', 0.979, 0.007, 0.703, 0.105, 0.762, 0.079],
   ['wdbscan', 0.982, 0.004, 0.983, 0.003, 0.351, 0.07],
   ['exactSTORM', 0.983, 0.004, 0.984, 0.003, 0.381, 0.078],
   ['COD', 0.978, 0.004, 0.718, 0.083, 0.356, 0.074],
   ['rrcf', 0.988, 0.004, 0.871, 0.065, 0.756, 0.079],
   ['ods', 0.986, 0.003, 0.944, 0.031, 0.541, 0.078]]},
 {'dataset_name': 'tune',
  'scores': [['dbscan', 0.996, 0.002, 0.981, 0.021, 0.883, 0.049],
   ['lof', 0.996, 0.002, 0.985, 0.017, 0.891, 0.047],
   ['wdbscan', 0.99, 0.003, 0.99, 0.003, 0.696, 0.069],
   ['exactSTORM', 0.99, 0.003, 0.99, 0.003, 0.704, 0.068],
   ['COD', 0.988, 0.003, 0.892, 0.064, 0.712, 0.065],
   ['rrcf', 0.993, 0.003, 0.954, 0.046, 0.835, 0.059],
   ['ods', 0.988, 0.003, 0.987, 0.003, 0.635, 0.081]]}]

In [8]:
import json
with open('total_results_'+features_set+'_'+metrics+'.json', 'w') as f:
    json.dump(res, f, indent=2)